Podpis cyfrowy - Bartosz Chwilkowski

Obliczanie liczby losowej

In [1]:
import psutil
import mmh3
import os
import time


filename = "decimal.txt"
filename2 = "freq.txt"
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
filepath = os.path.join(downloads_folder, filename)
filepath = os.path.join(downloads_folder, filename2)


#generowanie liczb losowych
def get_cpu_frequency():
    freq = psutil.cpu_freq().current
    return round(freq, 2) #aktualnie częstotliwość zegara w MHz

def MurmurHash(key, seed):
    hash_values = mmh3.hash128(key, int(seed))
    return hash_values

def randNum(n):
    #lista liczb losowych
    random_sequence = []

    #for i in range(1024): #generowanie x liczb
    binary_array = [None] * 8
    counter = 0

        

    while counter <= n: #generowanie 8bit liczb
        freq1 = int(get_cpu_frequency())
        time.sleep(0.01)

        key = str(freq1) #string użyty jako klucz
            
        freq2 = int(get_cpu_frequency()) #używana jako ziarno
        time.sleep(0.01)

        #funkcja skrótu
        hashed = MurmurHash(key, freq2)

        #zapis do tablicy binarnej
        binary_array[counter - 1] = hashed & 1 #przypisanie LSB do tablicy
            
        counter+=1

        #konwersja na liczbę dziesiętną
        binary_string = ''.join(str(bit) for bit in binary_array)
        decimal_number = int(binary_string, base=2)


        
        random_sequence.append(decimal_number)
    return random_sequence


RSA

In [2]:
import random
import rsa

#pliki z kluczami
pubkey = "pub_key.txt"
privkey = "priv_key.txt"
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
filepath = os.path.join(downloads_folder, pubkey)
filepath = os.path.join(downloads_folder, privkey)

#RSA
def generate_rsa_key_pair(bit_length):
    
    rsa.randfunc = randNum #podlaczenie trng, zeby to on pelnil funkcje zrodla losowosci

    (private_key, public_key) = rsa.newkeys(bit_length) #wygenerowanie pary kluczy o okreslonej dlugosci

    #zapis kluczy do pliku
    with open(pubkey, 'w') as file:
        file.write(str(public_key))
    with open(privkey, 'w') as file:
        file.write(str(private_key))
    
    return private_key, public_key

#zaszyfrowanie wiadomosci
def encrypt_message(message, private_key):
    encode_message = message.encode() 
    encrypted_message = rsa.encrypt(encode_message, private_key)

    return encrypted_message

#zdeszyfrowanie wiadomosci
def decrypt_message(encrypted_message, public_key):
    decrypted_message = rsa.decrypt(encrypted_message, public_key)

    return decrypted_message.decode()

Podpis cyfrowy - realizacja / uzycie

In [3]:
import binascii
import hashlib

def hash_string(message):
    # Tworzenie obiektu hashującego SHA-256
    hash_object = hashlib.sha256()

    # Konwertowanie stringa na bajty i aktualizacja obiektu hashującego
    hash_object.update(message.encode('utf-8'))

    # Pobranie zahashowanej wartości jako szesnastkowy string
    hashed_string = hash_object.hexdigest()

    return hashed_string

bit_lenght = 1024
private_key_pem, public_key_pem = generate_rsa_key_pair(bit_lenght)

# Zaszyfruj wiadomość
message_1 = "Hello, world!"
message_2 = "Helvdhlo, world!"

message = hash_string(message_1)
message2 = hash_string(message_2)

encrypted_message = encrypt_message(message, private_key_pem)

# Odszyfruj wiadomość
decrypted_message = decrypt_message(encrypted_message, public_key_pem)

print("Original message:", message)
print("Original message2:", message2)
print("Decrypted message:", decrypted_message)

# Wyświetl zaszyfrowaną wiadomość w postaci szesnastkowej
encrypted_message_hex = binascii.hexlify(encrypted_message)
print("Encrypted message (hex):", encrypted_message_hex)


#sprawdzenie kluczy
print("SPRAWDZENIE HASHOW")
if decrypted_message == message2:
    print("The hashed messages are the same.")
else:
    print("The hashed messages are different.")

Original message: 315f5bdb76d078c43b8ac0064e4a0164612b1fce77c869345bfc94c75894edd3
Original message2: 17917df997f039476f4395457bb62dc0819b11219051ca9f50ba24b865358baa
Decrypted message: 315f5bdb76d078c43b8ac0064e4a0164612b1fce77c869345bfc94c75894edd3
Encrypted message (hex): b'6a628d6e48ad6a84ca8946f9f8b7cc1f135d9463280574279ed38da96ab9f0559002f6bb03d293909bfc6582b508d131dbbbc679313434e939bf10ce86527770e269148f96d710a8c0267ad50a7ad7df128d5c7871574d1f5a2e8bbb7bd7fb26cfb792d1e1dd095780e79514b3fa3249cbabf64fb2152400b80e4e24876798ed'
SPRAWDZENIE HASHOW
The hashed messages are different.


Proba zdeszyfrowania wiadomosci za pomoca nowo wygenerowanego klucza

In [4]:
from rsa import DecryptionError


new_private_key, new_public_key = generate_rsa_key_pair(bit_lenght)

encrypted_message = encrypt_message(message, private_key_pem)


decrypted_message1 = decrypt_message(encrypted_message, new_public_key)
 

decrypted_message_new = decrypt_message(encrypted_message, public_key_pem)
print("Decrypted message using orginal private key:", decrypted_message_new)

print("SPRAWDZENIE HASHOW")
if decrypted_message1 == message2:
    print("The hashed messages are the same.")
else:
    print("The hashed messages are different.")


DecryptionError: Decryption failed